## Start of the project.
### Initial data inspection.

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

data = pd.read_csv('/datasets/data.csv')
data['education'] = data['education'].str.lower() 
data.info()

#data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


We import the libraries pandas, pymystem3, and collections, which we will need later.

Arter that we create a variable data in which we place the table with data, which we read using the .read_csv method.

For convenience, we convert the data in the 'education' column to lowercase so as not to be too distracting during the data analysis process.

We output general information about the data in the file. We observe pronounced discrepancies in the number of entries for 2 columns.

We display the first 10 rows on the screen to get an overall picture.

## Data Preprocessing.

### Handling gaps in table data.

In [2]:
data['days_employed'].isna().sum()
data['total_income'].isna().sum()


data_whith_none = data.loc[(data['days_employed'] == '') & (data['total_income'] == '')].count()

data = data.drop_duplicates()
data.groupby('income_type')['total_income'].describe()

data['days_employed'] = data['days_employed'].fillna(value = 0)
data['total_income'] = data['total_income'].fillna(value = 0)

data['family_status_id'].describe()
data['family_status_id'].value_counts()

data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)
data['dob_years'] = data['dob_years'].replace(0, data['dob_years'].mean())


def days_employed_fill (row):
    income_type_row = row['income_type']
    days_employed_row = row['days_employed']
    if income_type_row == 'пенсионер' and days_employed_row == 0:
        return 365213.306266
    elif income_type_row == 'студент' and days_employed_row == 0:
        return -578.751554
    elif income_type_row == 'безработный' and days_employed_row == 0:
        return 366413.652744
    elif income_type_row == 'в декрете' and days_employed_row == 0:
        return -3296.759962
    elif income_type_row == 'госслужащий' and days_employed_row == 0:
        return -2689.368353
    elif income_type_row == 'компаньон' and days_employed_row == 0:
        return 172357.950966
    elif income_type_row == 'предприниматель' and days_employed_row == 0:
        return -520.848083
    elif income_type_row == 'сотрудник' and days_employed_row == 0:
        return -1574.202821
    else:
        return days_employed_row

data['days_employed'] = data.apply(days_employed_fill ,axis = 1)

def total_income_fill (row):
    income_type_row = row['income_type']
    total_income_row = row['total_income']
    if income_type_row == 'пенсионер' and total_income_row == 0:
        return 118514.486412
    elif income_type_row == 'студент' and total_income_row == 0:
        return 98201.625314
    elif income_type_row == 'безработный' and total_income_row == 0:
        return 131339.751676
    elif income_type_row == 'в декрете' and total_income_row == 0:
        return 53829.130729
    elif income_type_row == 'госслужащий' and total_income_row == 0:
        return 150447.935283
    elif income_type_row == 'компаньон' and total_income_row == 0:
        return 172357.950966
    elif income_type_row == 'предприниматель' and total_income_row == 0:
        return 499163.144947
    elif income_type_row == 'сотрудник' and total_income_row == 0:
        return 142594.396847
    else:
        return total_income_row

data['total_income'] = data.apply(total_income_fill, axis = 1)


#print(2174 / 21525 *100)

data['gender'].max()

data['gender'].loc[data['gender'] == 'XNA'].count()



#data['days_employed'].describe()

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


1

Conclusion:
•  In both columns ('days_employed' and 'total_income'), there are 2174 missing values. Using the .loc attribute, which is commented out later, it was revealed that these are the same rows.

•  To fill in the empty rows in these two columns, median values were chosen because the average of 172 years of employment looks quite implausible.

•  The sequence of actions proposed in the plan was slightly violated, and duplicates were removed at this stage. This was done so that subsequent replacement of missing values with average or median values would not distort the final data (the number of duplicates is 71).

•  Empty values in the columns ('days_employed' and 'total_income') were filled with median values in order not to lose 10.09% of the data in case of their complete removal.

### Data type replacement

In [3]:
data['total_income'] = data['total_income'].astype(int)
data['days_employed'] = data['days_employed'].astype(int)
data['dob_years'] = data['dob_years'].astype(int)


data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


Conclusion:
•  Unusual cases across several columns were identified using the "min, max, and unique" methods.

•  There are 101 instances of zero values in the age column; the number of children as -1 and 20 (47 and 76 instances respectively); one instance of an unclear gender "XNA".


•  The gaps in the columns for employment duration and monthly income were filled with zero values.

•  After that, two functions were written, which, based on the employment type column, fill in the zero values in the columns with 50% of the sample values (identified earlier using the describe() method).

•  The .describe() and .value_counts() methods revealed zero values in the clients' age column (replaced with the average age as it does not participate in further calculations), the number of children "-1" was replaced with "1", and "20" with "2" (presumably an error related to human factors was made).

### Duplicate processing

In [4]:
data.duplicated().sum()

0

Duplicates were processed earlier in the 'Data Preprocessing' section.

A count of duplicates in the table was performed (value "0").

### Lemmatization.

We perform lemmatization on the 'purpose' column to standardize the words using the text_for_lemmatize function, which takes a string as an argument. It then returns the column value.

We add a new column to the main data table to record the lemmatized 'purpose' column. The argument is the lemmatization function mentioned above.

In [5]:
def text_for_lemmatize(row):
    return m.lemmatize(row['purpose'])  

data['lemmas'] = data.apply(text_for_lemmatize, axis=1) 

### Data Categorization

In [6]:
def purpose_category(row):   
    lemmas = row['lemmas']
    if 'жилье' in lemmas or 'недвижимость' in lemmas or 'квартира' in lemmas:
        return 'недвижимость'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'образование' in lemmas:
        return 'образование'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    
data['purpose_category'] = data.apply(purpose_category, axis = 1)

def total_income_category(row):
    row = row['total_income']
    if row < 145000:
        return 'ниже среднего'
    elif 145001 < row < 300000:
        return 'средний'
    else:
        return 'выше среднего'
    
data['total_income_category'] = data.apply(total_income_category, axis = 1)  


def children_category (row):  
    row = row['children']
    if row == 0:
        return 'без детей'
    elif 1 <= row <= 2:
        return 'семья с детьми'
    elif 3 <= row < 6:
        return 'многодетные'
    else:
        return 'over многодетные'
    
data['children_category'] = data.apply(children_category, axis = 1) 



Writing a function to categorize a new column with lemmas. The categories were identified earlier using the .unique() method.

Adding a column to the general table with categories for the purposes of obtaining a loan. The argument is a function for categorizing previously obtained lemmas.

A function for categorizing income level. Three categories: 'below average,' 'average,' and 'above average,' were chosen by me as the most commonly encountered for describing incomes. The initial threshold value was chosen based on the median value with a step of X2.

Adding a column to the general table with data with categories of income levels. The argument is a function for categorizing the income level.

Writing a function to break down the number of children into categories.

In [7]:
data['total_income_category'].value_counts()

ниже среднего    11130
средний           8841
выше среднего     1483
Name: total_income_category, dtype: int64

### Answers to questions posed to me before completing the project

1. Is there a relationship between having children and repaying the loan on time?

In [8]:
data_pivot_children = data.pivot_table(index = ['children_category'], columns='debt', values= 'total_income', aggfunc= 'count')

data_pivot_children['percentage_return'] = data_pivot_children[0] / (data_pivot_children[0] + data_pivot_children[1]) * 100

print(data_pivot_children)

debt                   0     1  percentage_return
children_category                                
без детей          13028  1063          92.456178
многодетные          349    31          91.842105
семья с детьми      6336   647          90.734641


### Conclusion

Constructing a pivot table to determine the relationship between having children and timely repayment of loans. Grouping by the column with categories of the number of children, the column with values indicating who repaid the loan on time and who did not, values - the number of people by the income column.

Adding to the pivot table a column with the percentage ratio of cases of timely money return to the total number by the category of having children.

The highest percentage of repayment is among families without children because it is much easier for them to manage the household budget and, as a consequence, to repay the loan on time.

Families with a large number of children, although they apply for loans less frequently, also have the lowest percentage of on-time repayment.

2. Is there a relationship between marital status and repayment of the loan on time?"


In [9]:
data_family_status = data.pivot_table(index = ['family_status_id', 'family_status'], columns = 'debt', values= 'total_income', aggfunc='count')

data_family_status['percentage_return'] = data_family_status[0] / (data_family_status[0] + data_family_status[1]) * 100

data_family_status

,debt,0,1,percentage_return
family_status_id,family_status,,,
0,женат / замужем,11408,931,92.454818
1,гражданский брак,3763,388,90.652855
2,вдовец / вдова,896,63,93.430657
3,в разводе,1110,85,92.887029
4,Не женат / не замужем,2536,274,90.249110


### Conclusion

Constructing a pivot table to determine the relationship between marital status and timely repayment of loans. Grouping by columns with family status ID and the family status itself, the column with values indicating who repaid the loan on time and who did not, values - the number of people by the income column.

Adding to the pivot table a column with the percentage ratio of cases of timely money return to the total number by the category of marital status.

The highest percentage of loan repayment was shown by the category 'widower / widow'. Judging by the total number of loan applications from this group of people, they need financial resources less than others and, as a consequence, experience the least problems with their repayment.

3. Is there a relationship between income level and repayment of the loan on time?

In [10]:
data_total_income_category = data.pivot_table(index=['total_income_category'], columns='debt', values='total_income', aggfunc='count')

data_total_income_category['percentage_return'] = data_total_income_category[0] / (data_total_income_category[0] + data_total_income_category[1]) * 100
data_total_income_category.sort_values('percentage_return', ascending= False)

debt,0,1,percentage_return
total_income_category,,,
выше среднего,1377,106,92.852326
средний,8143,698,92.104966
ниже среднего,10193,937,91.581312


### Conclusion

Constructing a pivot table to determine the relationship between income level and timely repayment of loans. Grouping by the column with income level categories, the column with values indicating who repaid the loan on time and who did not, values - the number of people by the income column.

The highest percentage was shown by the category of people with 'above average' income levels.

I assume that people with high income levels find it easier to find funds to repay loans compared to the other two categories. Linear progression confirms this.

## Final conclusion on the project

Based on the calculations for the given questions, the following conclusions can be drawn:

•  The level of income of the borrower has the least impact on the timely repayment of the loan.

•  The greatest discrepancy within the defined categories was shown by the metric with marital status.

•  In none of the categories was there a case of failure to repay the loan on time with a probability lower than 89.47%.